In [12]:
from sklearn.neighbors import KNeighborsClassifier
import os
os.chdir(os.path.join(os.getcwd(), '..','..', '..'))
from dataset.preprocessing.tf_idf_all_feature_matrix_gen import TFIDFFeatureGeneration
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier
import csv
import numpy as np
import pandas as pd

In [13]:
class KNN:
    def __init__(self, x_train, y_train, x_test, y_test, n, k):
        self.knn = KNeighborsClassifier(n_neighbors=n)
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.k = k

    def feature_selection_SelectKBest(self):
        k_best = SelectKBest(chi2, k=self.k)
        k_best.fit(self.x_train, self.y_train)
        self.x_train = k_best.transform(self.x_train)
        self.x_test = k_best.transform(self.x_test)
        
    def feature_selection_ExtraTreesClassifier(self):
        clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
        clf.fit(self.x_train, self.y_train)
        importances = clf.feature_importances_
        indices = np.argsort(importances)[::-1]
        self.x_train = self.x_train[:, indices[:self.k]]
        self.x_test = self.x_test[:, indices[:self.k]]

    def train(self):
        self.knn.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.knn.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        return f1_macro, f1_micro

In [14]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']
column_headings = ["Morbidity Class", "KNN1_Macro F1", "KNN1_Micro F1", "KNN5_Macro F1", "KNN5_Micro F1"]

In [15]:
with open("./results/tf-idf-features/performance_KNN_AllFeatures.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2], column_headings[3], column_headings[4]])

all_f1_macro1_scores = []
all_f1_micro1_scores = []

all_f1_macro5_scores = []
all_f1_micro5_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()

    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list1 = []
    f1_micro_list1 = []
    f1_macro_list5 = []
    f1_micro_list5 = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training KNN using TF-IDF Representation
        knn1_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 1, 100)
        knn1_obj.train()

        f1_macro1, f1_micro1 = knn1_obj.test_and_evaluate()

        f1_macro_list1.append(f1_macro1)
        f1_micro_list1.append(f1_micro1)

        knn5_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 5, 100)
        knn5_obj.train()

        f1_macro5, f1_micro5 = knn5_obj.test_and_evaluate()

        f1_macro_list5.append(f1_macro5)
        f1_micro_list5.append(f1_micro5)

    f1_macro1 = np.mean(f1_macro_list1)
    f1_micro1 = np.mean(f1_micro_list1)
    f1_macro5 = np.mean(f1_macro_list5)
    f1_micro5 = np.mean(f1_micro_list5)

    print(f"For n=1, Macro F1 score: {f1_macro1} and Micro F1 Score {f1_micro1}")
    print(f"For n=5, Macro F1 score: {f1_macro5} and Micro F1 Score {f1_micro5}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro1, f1_micro1, f1_macro5, f1_micro5]
    all_f1_macro1_scores.append(f1_macro1)
    all_f1_micro1_scores.append(f1_micro1)

    all_f1_macro5_scores.append(f1_macro5)
    all_f1_micro5_scores.append(f1_micro5)


    with open("./results/tf-idf-features/performance_KNN_AllFeatures.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/tf-idf-features/performance_KNN_AllFeatures.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([
        sum(all_f1_macro1_scores)/len(all_f1_macro1_scores),  sum(all_f1_micro1_scores)/len(all_f1_micro1_scores),
        sum(all_f1_macro5_scores)/len(all_f1_macro5_scores),  sum(all_f1_micro5_scores)/len(all_f1_micro5_scores) 
                ])
    writer.writerow(row)

Asthma
(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
For n=1, Macro F1 score: 0.5892440024744311 and Micro F1 Score 0.8145795523290985
For n=5, Macro F1 score: 0.5682579274393165 and Micro F1 Score 0.8759225650332729
CAD
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
For n=1, Macro F1 score: 0.6159731039716542 and Micro F1 Score 0.6586195286195287
For n=5, Macro F1 score: 0.63267944156766 and Micro F1 Score 0.6879797979797979
CHF
(243, 600) (243,) Counter({1.0: 243})
For n=1, Macro F1 score: 1.0 and Micro F1 Score 1.0
For n=5, Macro F1 score: 1.0 and Micro F1 Score 1.0
Depression
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
For n=1, Macro F1 score: 0.5676511850007012 and Micro F1 Score 0.7113383985973115
For n=5, Macro F1 score: 0.5884863629899829 and Micro F1 Score 0.7730566919929867
Diabetes
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
For n=1, Macro F1 score: 0.5484186689831109 and Micro F1 Score 0.6473997493734337
For n=5, Macro F1 score: 0.5444431574041595 and Micro F1 Sco

In [16]:
with open("./results/tf-idf-features/performance_KNN_SelectKBest.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2], column_headings[3], column_headings[4]])

all_f1_macro1_scores = []
all_f1_micro1_scores = []

all_f1_macro5_scores = []
all_f1_micro5_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()

    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list1 = []
    f1_micro_list1 = []
    f1_macro_list5 = []
    f1_micro_list5 = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training KNN using TF-IDF Representation
        knn1_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 1, 100)
        knn1_obj.feature_selection_SelectKBest()
        knn1_obj.train()

        f1_macro1, f1_micro1 = knn1_obj.test_and_evaluate()

        f1_macro_list1.append(f1_macro1)
        f1_micro_list1.append(f1_micro1)

        knn5_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 5, 100)
        knn5_obj.feature_selection_SelectKBest()
        knn5_obj.train()

        f1_macro5, f1_micro5 = knn5_obj.test_and_evaluate()

        f1_macro_list5.append(f1_macro5)
        f1_micro_list5.append(f1_micro5)

    f1_macro1 = np.mean(f1_macro_list1)
    f1_micro1 = np.mean(f1_micro_list1)
    f1_macro5 = np.mean(f1_macro_list5)
    f1_micro5 = np.mean(f1_micro_list5)

    print(f"For n=1, Macro F1 score: {f1_macro1} and Micro F1 Score {f1_micro1}")
    print(f"For n=5, Macro F1 score: {f1_macro5} and Micro F1 Score {f1_micro5}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro1, f1_micro1, f1_macro5, f1_micro5]
    all_f1_macro1_scores.append(f1_macro1)
    all_f1_micro1_scores.append(f1_micro1)

    all_f1_macro5_scores.append(f1_macro5)
    all_f1_micro5_scores.append(f1_micro5)


    with open("./results/tf-idf-features/performance_KNN_SelectKBest.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/tf-idf-features/performance_KNN_SelectKBest.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([
        sum(all_f1_macro1_scores)/len(all_f1_macro1_scores),  sum(all_f1_micro1_scores)/len(all_f1_micro1_scores),
        sum(all_f1_macro5_scores)/len(all_f1_macro5_scores),  sum(all_f1_micro5_scores)/len(all_f1_micro5_scores) 
                ])
    writer.writerow(row)

Asthma
(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
For n=1, Macro F1 score: 0.5585601368072391 and Micro F1 Score 0.8268300060496067
For n=5, Macro F1 score: 0.5125141298277824 and Micro F1 Score 0.8810949788263762
CAD
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
For n=1, Macro F1 score: 0.6430374695646784 and Micro F1 Score 0.6532659932659933
For n=5, Macro F1 score: 0.6859077413744469 and Micro F1 Score 0.6971043771043771
CHF
(243, 600) (243,) Counter({1.0: 243})
For n=1, Macro F1 score: 1.0 and Micro F1 Score 1.0
For n=5, Macro F1 score: 1.0 and Micro F1 Score 1.0
Depression
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
For n=1, Macro F1 score: 0.5667215910734993 and Micro F1 Score 0.7216540035067213
For n=5, Macro F1 score: 0.5400134021248935 and Micro F1 Score 0.7748392752776155
Diabetes
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
For n=1, Macro F1 score: 0.6154540769219247 and Micro F1 Score 0.6825501253132832
For n=5, Macro F1 score: 0.6380515625246815 and Micro F1 S

In [17]:
with open("./results/tf-idf-features/performance_KNN_ExtraTreesClassifier.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2], column_headings[3], column_headings[4]])

all_f1_macro1_scores = []
all_f1_micro1_scores = []

all_f1_macro5_scores = []
all_f1_micro5_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()

    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list1 = []
    f1_micro_list1 = []
    f1_macro_list5 = []
    f1_micro_list5 = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training KNN using TF-IDF Representation
        knn1_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 1, 100)
        knn1_obj.feature_selection_ExtraTreesClassifier()
        knn1_obj.train()

        f1_macro1, f1_micro1 = knn1_obj.test_and_evaluate()

        f1_macro_list1.append(f1_macro1)
        f1_micro_list1.append(f1_micro1)

        knn5_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 5, 100)
        knn5_obj.feature_selection_ExtraTreesClassifier()
        knn5_obj.train()

        f1_macro5, f1_micro5 = knn5_obj.test_and_evaluate()

        f1_macro_list5.append(f1_macro5)
        f1_micro_list5.append(f1_micro5)

    f1_macro1 = np.mean(f1_macro_list1)
    f1_micro1 = np.mean(f1_micro_list1)
    f1_macro5 = np.mean(f1_macro_list5)
    f1_micro5 = np.mean(f1_micro_list5)

    print(f"For n=1, Macro F1 score: {f1_macro1} and Micro F1 Score {f1_micro1}")
    print(f"For n=5, Macro F1 score: {f1_macro5} and Micro F1 Score {f1_micro5}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro1, f1_micro1, f1_macro5, f1_micro5]
    all_f1_macro1_scores.append(f1_macro1)
    all_f1_micro1_scores.append(f1_micro1)

    all_f1_macro5_scores.append(f1_macro5)
    all_f1_micro5_scores.append(f1_micro5)


    with open("./results/tf-idf-features/performance_KNN_ExtraTreesClassifier.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/tf-idf-features/performance_KNN_ExtraTreesClassifier.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([
        sum(all_f1_macro1_scores)/len(all_f1_macro1_scores),  sum(all_f1_micro1_scores)/len(all_f1_micro1_scores),
        sum(all_f1_macro5_scores)/len(all_f1_macro5_scores),  sum(all_f1_micro5_scores)/len(all_f1_micro5_scores) 
                ])
    writer.writerow(row)

Asthma
(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
For n=1, Macro F1 score: 0.5278217949096919 and Micro F1 Score 0.8688747731397459
For n=5, Macro F1 score: 0.505398472032971 and Micro F1 Score 0.8828493647912886
CAD
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
For n=1, Macro F1 score: 0.6824731264416342 and Micro F1 Score 0.6935353535353536
For n=5, Macro F1 score: 0.6968978475281808 and Micro F1 Score 0.7079124579124579
CHF
(243, 600) (243,) Counter({1.0: 243})
For n=1, Macro F1 score: 1.0 and Micro F1 Score 1.0
For n=5, Macro F1 score: 1.0 and Micro F1 Score 1.0
Depression
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
For n=1, Macro F1 score: 0.5423872024531897 and Micro F1 Score 0.7578609000584453
For n=5, Macro F1 score: 0.5069387078275198 and Micro F1 Score 0.7869666861484512
Diabetes
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
For n=1, Macro F1 score: 0.6568890734896928 and Micro F1 Score 0.725062656641604
For n=5, Macro F1 score: 0.6044734403546814 and Micro F1 Sco